In [2]:
import tensorflow as tf
import tensorflow.examples.tutorials.mnist.input_data as input_data

In [3]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


## 建立模型

In [4]:
# define the fullly connect network
def fcn_layer(inputs,
             input_dim,
             output_dim,
             activation=None):
    W = tf.Variable(tf.truncated_normal([input_dim, output_dim], stddev=0.1))
    b = tf.Variable(tf.zeros([output_dim]))
    XWB = tf.matmul(inputs, W) + b
    if activation == None:
        return XWB
    return activation(XWB)    

## 构建输入层

In [5]:
x = tf.placeholder(tf.float32, [None, 28*28], name="X")

## 构建隐藏层

In [7]:
# hidden layer contains 256 nueral
h1=fcn_layer(inputs=x,
             input_dim=28*28,
             output_dim=256,
             activation=tf.nn.relu)


Instructions for updating:
Colocations handled automatically by placer.


## 构建输出层

In [8]:
forward = fcn_layer(inputs=h1,
             input_dim=256,
             output_dim=10,
             activation=None)

pred = tf.nn.softmax(forward)

## 训练模型
### 定义标签

In [9]:
y = tf.placeholder(tf.float32, [None, 10], name = "Y")

### 定义损失函数

In [11]:
loss_function = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=forward))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



### 设置训练参数

In [13]:
train_epochs = 40
batch_size = 50
total_batch = mnist.train.num_examples // batch_size
display_step = 1
learning_rate = 0.01

### 设置优化器

In [14]:
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss_function)

### 定义准确率

In [15]:
correct_predict = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_predict, tf.float32))

###  训练模型

In [17]:
from time import time
startTime = time()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(train_epochs):
        for batch in range(total_batch):
            xs, ys = mnist.train.next_batch(batch_size)
            sess.run(optimizer, feed_dict={x: xs, y: ys})
            
        loss, acc=sess.run([loss_function, accuracy],
                           feed_dict={x:mnist.validation.images,
                                     y:mnist.validation.labels})
        
        if epoch % display_step == 0:
            print("epoch: %02d" % epoch, 
                  "Loss: {:.9f}".format(loss),
                  "Acc: {:.9f}".format(acc))

epoch: 00 Loss: 0.159955770 Acc: 0.950999975
epoch: 01 Loss: 0.160867453 Acc: 0.954400003
epoch: 02 Loss: 0.143644154 Acc: 0.964200020
epoch: 03 Loss: 0.183215335 Acc: 0.957000017
epoch: 04 Loss: 0.197829440 Acc: 0.960799992
epoch: 05 Loss: 0.193925411 Acc: 0.964200020
epoch: 06 Loss: 0.176604524 Acc: 0.966600001
epoch: 07 Loss: 0.202888712 Acc: 0.968999982
epoch: 08 Loss: 0.205953315 Acc: 0.969200015
epoch: 09 Loss: 0.256676257 Acc: 0.967199981
epoch: 10 Loss: 0.209435552 Acc: 0.970799983
epoch: 11 Loss: 0.215416357 Acc: 0.971800029
epoch: 12 Loss: 0.245943725 Acc: 0.971599996
epoch: 13 Loss: 0.255851209 Acc: 0.972999990
epoch: 14 Loss: 0.269353181 Acc: 0.970200002
epoch: 15 Loss: 0.333265811 Acc: 0.967599988
epoch: 16 Loss: 0.258328259 Acc: 0.972400010
epoch: 17 Loss: 0.312843859 Acc: 0.972999990
epoch: 18 Loss: 0.402283877 Acc: 0.968800008
epoch: 19 Loss: 0.371536911 Acc: 0.969799995
epoch: 20 Loss: 0.306877434 Acc: 0.975000024
epoch: 21 Loss: 0.310533941 Acc: 0.972999990
epoch: 22 